In [1]:
!pip -q install ultralytics

In [2]:
!wget https://the-eye.eu/public/AI/cah/laion400m-met-release/laion400m-meta/part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet

--2024-02-16 00:50:38--  https://the-eye.eu/public/AI/cah/laion400m-met-release/laion400m-meta/part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
Resolving the-eye.eu (the-eye.eu)... 162.213.130.6
Connecting to the-eye.eu (the-eye.eu)|162.213.130.6|:443... connected.
^C


In [1]:
import json
import pandas as pd
import numpy as np
import urllib3
from tqdm import trange
import os
import torch
from ultralytics import YOLO
from collections import Counter
from scipy import stats
import requests
from datetime import datetime


In [2]:
metadata = pd.read_parquet("/content/part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet")
print(metadata.shape)
metadata = metadata[0:600000]

(12933524, 8)


In [3]:
model = YOLO("yolov8n.pt")
IMG_FORMATS = ['bmp', 'dng', 'jpeg', 'jpg', 'mpo', 'png', 'tif', 'tiff', 'webp', 'pfm']
urls_raw = list(metadata["URL"])
caps_raw = list(metadata["TEXT"])

In [4]:
metadata[0:1]

,SAMPLE_ID,URL,TEXT,HEIGHT,WIDTH,LICENSE,NSFW,similarity
0,1.581282e+12,http://media.rightmove.co.uk/148k/147518/58718...,View EPC Rating Graph for this property,109.0,100.0,?,UNSURE,0.312813


In [5]:
import re
def cap_parse(cap):
    nums = []
    num2word = {
        'two':2,
        'three':3,
        'four':4,
        'five':5,
        'six':6,
        'seven':7,
        'eight':8,
        'nine':9,
        'ten':10
    }
    try:
        x = re.findall(r'\b(two|three|four|five|six|seven|eight|nine|ten)\b', cap, flags=re.IGNORECASE)
    except:
        return (False,-1)
    if len(x) == 0 :return (False,-1)
    else:
        max_freq = max(num2word[i.lower()] for i in x)
        return (True,max_freq)



txt0 = "vintage silver plate tablespoons, serving spoon set of six 1847 Rogers pattern"
txt1 = "A preview of seven printable dot marker coloring pages. Each page has a large 4th of July themed image with dots to color in with a dauber style marker."
txt2 = "bag of chips"
cap_parse(txt2)


(False, -1)

In [6]:

def validate_urls(urls,caps):
    valid_urls = []
    valid_caps = []
    for url,cap in zip(urls,caps):
        for img_format in IMG_FORMATS:
            idx = url.lower().find("." + img_format)
            if idx == -1:
                continue
            valid_urls.append(url[:idx + len(img_format) + 1])
            valid_caps.append(cap)
    return (valid_urls,valid_caps)




In [7]:
def validate_counts(counting):
    final = []
    for i in range(len(counting)):
        print(counting[i])
        x,cap,url,num = counting[i]
        if len(x)!=0:
            counts = Counter(map(int, x))
            max_freq = max(counts.values())
            print(max_freq)
            if max_freq == num:
                final.append(counting[i])
    return final


In [8]:
a = [([torch.tensor(39., dtype=int), torch.tensor(39., dtype=int)],"aa","bb",3),([39,39,39],"aa","bb",2)]
final = validate_counts(a)
final
# a[0]
# x,cap,url,num = a[0]

([tensor(39), tensor(39)], 'aa', 'bb', 3)
2
([39, 39, 39], 'aa', 'bb', 2)
3


[]

In [9]:
validate_counts([([torch.tensor(32), torch.tensor(32), torch.tensor(0)], 'http://s3./bncore/wp-conten.jpg', 'Matthew Anderson', 5)])

([tensor(32), tensor(32), tensor(0)], 'http://s3./bncore/wp-conten.jpg', 'Matthew Anderson', 5)
2


[]

In [10]:
def save_counts(counting):
    print(len(counting))
    counting[0:5]
    f = open("segments0.npy","wb")
    np.save(f,pd.DataFrame(counting))
    print("counting set saved")


In [11]:
# import sys, os

# def blockPrint():
#     sys.stdout = open(os.devnull, 'w')

# # Restore
# def enablePrint():
#     sys.stdout = sys.__stdout__


In [12]:
# def verify_url(url, timeout=1):
#     try:
#         start_time = datetime.now()
#         response = requests.get(url, timeout=timeout)
#         end_time = datetime.now()
#         time_taken = (end_time - start_time).total_seconds()
#         if response.status_code == 200 and time_taken <= timeout:
#             # print(f"URL {url} is reachable and took {time_taken} seconds to fetch.")
#             return True
#         else:
#             # print(f"URL {url} is unreachable or took more than {timeout} seconds to fetch.")
#             return False
#     except:
#         pass

In [13]:

def filter_laion(urls_raw,caps_raw):
    urls,caps = validate_urls(urls_raw,caps_raw)
    counting = []
    timeout = 1
    print(len(urls))
    for i in trange(len(urls)):
        # blockPrint()
        cap_data = cap_parse(caps[i])
        # print(cap_data,caps[i])
        if cap_data[0] == True:
            try:
                start_time = datetime.now()
                response = requests.get(urls[i], timeout=timeout)
                end_time = datetime.now()
                time_taken = (end_time - start_time).total_seconds()

                if response.status_code == 200 and time_taken <= timeout:
                  results = model(str(urls[i]),verbose=False,stream=False)
                  x = list(results[0].boxes.cls)
                else:
                  x=[]
            except:
                x = []
            counting.append((x,urls[i],caps[i],cap_data[1]))
        # enablePrint()
    counting = validate_counts(counting)
    save_counts(counting)
    return counting



In [ ]:
filter_laion(urls_raw,caps_raw)



In [ ]:
f = open("segments0.npy","rb")
out = np.load(f,allow_pickle=True)
len(out)

In [ ]:
out

In [ ]:
validate_counts([(list([torch.tensor(39.),torch.tensor(39.),torch.tensor(39.)]),"aa","bb",3)])

In [ ]:
import requests

def is_url_image(image_url):
   image_formats = ("image/png", "image/jpeg", "image/jpg")
   try:
      r = requests.head(image_url)
   except:
      return False
   try:
      if r.headers["content-type"] in image_formats:
         return True
   except:
         return True

   return False